# Computing a matrix of Levenshtein distances

In [ ]:
// Set up notebook:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)


In [ ]:
// Build data
import scala.io.Source
// construct URLs for the five documents:
val names = Vector("bancroft", "bliss", "everett", "hay", "nicolay")
val baseUrl = "https://raw.githubusercontent.com/neelsmith/gettysburg/master/texts/"
val urls = names.map( f => baseUrl + f + ".txt")

val texts = urls.map(u => Source.fromURL(u).getLines.mkString("\n"))
val tidier = texts.map(t => t.replaceAll( "[\\.,\\-]", ""))

In [ ]:
val corpus = tidier.map(t => t.split("\n").toVector.filter(_.nonEmpty))


In [ ]:
// get corresponding passage from all texts
def extractPassage(column: Int) : Vector[String] = {
  corpus.map(row => {
    if (column >= row.size) {
      ""
    } else {
      row(column)
    }})
}


In [ ]:
// Edit distance using Levenshtein method
import scala.collection.mutable
import scala.collection.parallel.ParSeq

// Implementation from RosettaCode:
// https://rosettacode.org/wiki/Levenshtein_distance
def levenshteinMemo(s1: String, s2: String): mutable.Map[(Int, Int), Int] = {
  val memoizedCosts = mutable.Map[(Int, Int), Int]()

  def lev: ((Int, Int)) => Int = {
    case (k1, k2) =>
      memoizedCosts.getOrElseUpdate((k1, k2), (k1, k2) match {
        case (i, 0) => i
        case (0, j) => j
        case (i, j) =>
          ParSeq(1 + lev((i - 1, j)),
                 1 + lev((i, j - 1)),
                 lev((i - 1, j - 1))
                   + (if (s1(i - 1) != s2(j - 1)) 1 else 0)).min
      })
  }
  lev((s1.length, s2.length))
  memoizedCosts
}

def editDistance(s1: String, s2: String) : Int = {
  levenshteinMemo(s1, s2)((s1.length, s2.length))
}

In [ ]:
//
// @param baseLine Text of line unit
// @param cfTexts list of corresponding unit in all texts
def rowData(baseLine: String, cfTexts: Vector[String]) : Vector[Int] = {
  val data = for (i <- 0 until cfTexts.size) yield {
    editDistance(baseLine, cfTexts(i))
  }
  data.toVector
}


In [ ]:

def dataMatrix = for (documentsIndex <- 0 until byLine.size) yield {
  println("Document index " + documentsIndex)
  val baseText = corpus(documentsIndex)
  //val cellCount = byLine.size * byLine.size
  val colName = names(documentsIndex)
  println(colName)

  val dataByLine = for (lineIndex <- 0 until baseText.size) yield {
    val baseTextPassage = baseText(lineIndex)
    val rowLabel = s"${colName}." + lineIndex
    val cfLines = extractPassage(lineIndex)

    println("Computing ed. distance from " + rowLabel)
    val data = rowData(baseTextPassage, cfLines)
    rowLabel + "," + data.mkString(",")
  }
  println("Done.")
  dataByLine
}


In [ ]:
val dm = dataMatrix

In [ ]:
val colLabels = "base," + names.mkString(",")

In [ ]:
println(colLabels + "\n" + dm.flatten.mkString("\n"))

In [ ]:
val banc = corpus.head
val bliss = corpus(2)

In [ ]:
banc(2)
bliss(2)
editDistance(banc(2), bliss(2))